# Лабораторная работа №4

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('data/diabetes.csv')
data


## Проведите предварительную обработку данных, включая обработку отсутствующих значений, кодирование категориальных признаков и масштабирование.


Проверим наличие пропущенных значений в датасете:

In [ ]:
print("Пропуски до обработки:\n", data.isnull().sum(), "\n")

Пропущенных значений нет.

Заменить нулевые значения в некоторых столбцах на NaN, так как нули в этих признаках означают отсутствие измерения.

In [ ]:
cols_with_zeros = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]

data[cols_with_zeros] = data[cols_with_zeros].replace(0, np.nan)

print("Пропуски после замены нулей на NaN:\n", data.isnull().sum(), "\n")

В нашем датасете все признаки числовые, кроме целевого Outcome, кодировать ничего не нужно.

## Получите и визуализируйте (графически) статистику по датасету (включая количество, среднее значение, стандартное отклонение, минимум, максимум и различные квантили), постройте 3d-визуализацию признаков.

Получим статистику по датасету:

In [ ]:
stats = data.describe()
print(stats)

Построим гистограммы и boxplot для всех признаков:

In [ ]:
data.hist(bins=20, figsize=(12,10))
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,6))
data.plot(kind='box')
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

x = data['Glucose']
y = data['BMI']
z = data['Age']

ax.scatter(x, y, z, c=data['Outcome'], cmap='viridis', alpha=0.7)
ax.set_xlabel('Glucose')
ax.set_ylabel('BMI')
ax.set_zlabel('Age')
plt.show()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

x = data['Pregnancies']
y = data['BMI']
z = data['Age']

ax.scatter(x, y, z, c=data['Outcome'], cmap='viridis', alpha=0.7)
ax.set_xlabel('Pregnancies')
ax.set_ylabel('BMI')
ax.set_zlabel('Age')
plt.show()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

x = data['Pregnancies']
y = data['Glucose']
z = data['Age']

ax.scatter(x, y, z, c=data['Outcome'], cmap='viridis', alpha=0.7)
ax.set_xlabel('Pregnancies')
ax.set_ylabel('Glucose')
ax.set_zlabel('Age')
plt.show()

Построим матрицу крорреляций признаков:

In [ ]:
corr_matrix = data.corr(numeric_only=True)

plt.figure(figsize=(10, 8))
plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Коэффициент корреляции')
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, ha='right')
plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns)
plt.title("Матрица корреляции признаков")
plt.tight_layout()
plt.show()

Выполним min-max масштабирование для всех признаков, кроме целевого Outcome:

$$ X_{\text{norm}} = \frac{X - X_{\min}}{X_{\max} - X_{\min}} $$

In [ ]:
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

X_norm = X.copy()
for col in X.columns:
    min_val = X[col].min()
    max_val = X[col].max()
    X_norm[col] = (X[col] - min_val) / (max_val - min_val)

data_norm = pd.concat([X_norm, y], axis=1)

print(data_norm.head())

Все значения в диапазоне [0, 1].

Получим статистику по нормализованному датасету:

In [ ]:
stats = data_norm.describe()
print(stats)

Построим гистограммы и boxplot для всех признаков нормализованного датасета:

In [ ]:
data_norm.hist(bins=20, figsize=(12, 10))
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
data_norm.plot(kind='box')
plt.tight_layout()
plt.show()

## Реализуйте метод k-ближайших соседей ****без использования сторонних библиотек, кроме NumPy и Pandas.


Перед обучением разделим данные на обучающую и тестовую выборки (80/20), выполним нормализацию на обучающей выборке и применим те же параметры нормализации к тестовой выборке.

Датасет разделяется на обучающую выборку $X_{\text{train}}, y_{\text{train}}$
и тестовую выборку $X_{\text{test}}, y_{\text{test}}$.
Разделение выполняется случайным образом с фиксированным генератором случайных чисел
для воспроизводимости результатов.

In [ ]:
X_df = data.drop('Outcome', axis=1).copy()
y_sr = data['Outcome'].astype(int).copy()

def train_test_split_df(X: pd.DataFrame, y: pd.Series, seed=42, test_size=0.2):
    n = len(X)
    rng = np.random.default_rng(seed)
    idx = np.arange(n)
    rng.shuffle(idx)
    cut = int(n * (1 - test_size))
    train_idx, test_idx = idx[:cut], idx[cut:]
    return (X.iloc[train_idx].reset_index(drop=True),
            X.iloc[test_idx].reset_index(drop=True),
            y.iloc[train_idx].reset_index(drop=True),
            y.iloc[test_idx].reset_index(drop=True))

def fit_minmax(X: pd.DataFrame):
    mins = X.min(axis=0)
    maxs = X.max(axis=0)
    denom = (maxs - mins).replace(0, 1.0)
    return mins, denom

def transform_minmax(X: pd.DataFrame, mins: pd.Series, denom: pd.Series):
    return (X - mins) / denom

X_train_df, X_test_df, y_train_sr, y_test_sr = train_test_split_df(X_df, y_sr, seed=42, test_size=0.2)
mins, denom = fit_minmax(X_train_df)
X_train_norm = transform_minmax(X_train_df, mins, denom)
X_test_norm  = transform_minmax(X_test_df,  mins, denom)

print("HEAD (train, norm):")
display(X_train_norm.head())
print("DESCRIBE (train, norm):")
display(X_train_norm.describe().T)

Для классификации нового объекта алгоритм ищет $k$ наиболее близких обучающих
примеров и относит данный объект к тому классу, который наиболее часто встречается среди этих соседей.

Пусть обучающая выборка состоит из $N$ объектов:
$$
\mathcal{D} = \{(x_1, y_1), (x_2, y_2), \dots, (x_N, y_N)\},
$$
где $x_i \in \mathbb{R}^m$ — вектор признаков, а $y_i \in \{0,1\}$ — метка класса.
Для нового объекта $x^*$ вычисляется расстояние до всех точек обучающей выборки.
В данной работе используется евклидово расстояние:

$$
d(x^*, x_i) = \sqrt{\sum_{j=1}^{m} (x_j^* - x_{ij})^2}.
$$

Затем выбираются индексы $k$ ближайших соседей:
$$
N_k(x^*) = \operatorname{arg\,min}_{i_1,\dots,i_k} d(x^*, x_{i_j}).
$$

Класс нового объекта определяется по большинству меток среди найденных соседей:
$$
\hat{y}(x^*) =
\begin{cases}
1, & \text{если } \frac{1}{k} \sum\limits_{i \in N_k(x^*)} y_i \ge 0.5,\\
0, & \text{иначе.}
\end{cases}
$$

Для оценки эффективности классификации используется метрика точность:
$$
\text{Accuracy} = \frac{1}{n_{\text{test}}}
\sum_{i=1}^{n_{\text{test}}} [y_i = \hat{y}_i],
$$
где $[y_i = \hat{y}_i] = 1$, если предсказание совпадает с истинной меткой, и $0$ иначе.
Эта метрика отражает долю правильно классифицированных примеров на тестовой выборке.

Для анализа ошибок классификации строится матрица ошибок:
$$
\begin{bmatrix}
\text{TP} & \text{FN} \\
\text{FP} & \text{TN}
\end{bmatrix},
$$
где:
- TP (True Positive) — число объектов класса 1, правильно классифицированных;
- TN (True Negative) — число объектов класса 0, правильно классифицированных;
- FP (False Positive) — объекты класса 0, ошибочно отнесённые к 1;
- FN (False Negative) — объекты класса 1, ошибочно отнесённые к 0.


In [ ]:
def knn_predict_batch(X_train_np, y_train_np, X_test_np, k=5):
    preds = np.empty(X_test_np.shape[0], dtype=int)
    for i, x in enumerate(X_test_np):
        dists = np.sqrt(np.sum((X_train_np - x) ** 2, axis=1))
        nn_idx = np.argpartition(dists, kth=k-1)[:k]
        labels = y_train_np[nn_idx]
        counts = np.bincount(labels)
        if (counts.max() != 0) and (np.sum(counts == counts.max()) == 1):
            preds[i] = np.argmax(counts)
        else:
            preds[i] = int(np.rint(labels.mean()))
    return preds

def accuracy_score(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=int)
    y_pred = np.asarray(y_pred, dtype=int)
    return (y_true == y_pred).mean()

def confusion_matrix_df(y_true, y_pred, labels=(0, 1)):
    ct = pd.crosstab(
        pd.Series(y_true, name='True').astype(pd.CategoricalDtype(categories=labels)),
        pd.Series(y_pred, name='Pred').astype(pd.CategoricalDtype(categories=labels)),
        dropna=False
    )
    return ct.reindex(index=labels, columns=labels, fill_value=0)

def run_knn_report(features, k=5, seed=42):
    X_train_df, X_test_df, y_train_sr, y_test_sr = train_test_split_df(X_df, y_sr, seed=seed, test_size=0.2)
    y_train_sr = y_train_sr.astype(int)
    y_test_sr = y_test_sr.astype(int)

    mins, denom = fit_minmax(X_train_df)
    X_train_norm = transform_minmax(X_train_df, mins, denom)
    X_test_norm  = transform_minmax(X_test_df,  mins, denom)

    cols = list(X_df.columns) if features == 'ALL' else list(features)

    Xtr = X_train_norm[cols].to_numpy(dtype=float)
    Xte = X_test_norm[cols].to_numpy(dtype=float)
    ytr = y_train_sr.to_numpy(dtype=int)
    yte = y_test_sr.to_numpy(dtype=int)

    print(f"Train size: {len(Xtr)}")
    print(y_train_sr.astype(float).value_counts().sort_index())
    print()
    print(f"Test size: {len(Xte)}")
    print(y_test_sr.astype(float).value_counts().sort_index())

    y_pred = knn_predict_batch(Xtr, ytr, Xte, k=k)
    print("\nОценка модели: ", accuracy_score(yte, y_pred))

    display(X_train_norm[cols].head())

    return y_pred, yte

def plot_confusion_matrix(cm_df, title="Confusion matrix"):
    fig, ax = plt.subplots(figsize=(4,4))
    im = ax.imshow(cm_df.values, cmap='Greens')
    ax.set_title(title)
    ax.set_xlabel('Pred'); ax.set_ylabel('True')
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(cm_df.columns); ax.set_yticklabels(cm_df.index)
    for (i, j), v in np.ndenumerate(cm_df.values):
        ax.text(j, i, int(v), ha='center', va='center')
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()

## Постройте две модели k-NN с различными наборами признаков. Для каждой модели проведите оценку на тестовом наборе данных при разных значениях k. Выберите несколько различных значений k. Постройте матрицу ошибок.

### Модель 1: Признаки случайно отбираются.

Выберем случайные признаки для модели и проведем оценку при k=3, k=5, k=10.

In [ ]:
rng = np.random.default_rng(10)
rand_feats_1 = list(rng.choice(X_df.columns, size=4, replace=False))
rand_feats_2 = list(rng.choice(X_df.columns, size=5, replace=False))
rand_feats_3 = list(rng.choice(X_df.columns, size=4, replace=False))
rand_feats = [rand_feats_1, rand_feats_2, rand_feats_3]

ks = [3, 5, 10]

for i, k in enumerate(ks):
    print(f"\n=== Случайные признаки: {rand_feats[i]}, k={k} ===")
    y_pred_fix, y_test_fix = run_knn_report(rand_feats[i], k=k, seed=42)
    cm_fix = confusion_matrix_df(y_test_fix, y_pred_fix, labels=(0,1))
    print("\nМатрица ошибок:")
    print(cm_fix)
    plot_confusion_matrix(cm_fix, title=f"Confusion matrix (random), k={k})")


### Модель 2: Фиксированный набор признаков, который выбирается заранее.

Выберем фиксированный набор признаков для модели и проведем оценку при k=3, k=5, k=10.

In [ ]:
fixed_features = ['Glucose', 'BMI', 'Age', 'BloodPressure']

for k in [3, 5, 10]:
    print(f"\n=== Фиксированные признаки: {fixed_features}, k={k} ===")
    y_pred_fix, y_test_fix = run_knn_report(fixed_features, k=k, seed=42)
    cm_fix = confusion_matrix_df(y_test_fix, y_pred_fix, labels=(0,1))
    print("\nМатрица ошибок:")
    print(cm_fix)
    plot_confusion_matrix(cm_fix, title=f"Confusion matrix (fixed, k={k})")
